In [1]:
import numpy as np
import os
import threading
import operator
import pickle
import re
import pandas as pd


start_time = 1190146243
end_time = 1192994591

def load_data(input_name):
    with open(input_name, 'rb') as f:
        data = pickle.load(f)
    return data
def dump_data(data, output_name):
    with open(output_name, 'wb') as f:
        pickle.dump(data, f)




import mwclient
from itertools import islice
import calendar
from urllib.parse import unquote
from urllib.parse import quote
import time

site = mwclient.Site('en.wikipedia.org')

def get_page(title):
    page = site.pages[unquote(unquote(title))]
    return page

# Get revision records

In [2]:
def get_revisions(title, prop='ids|timestamp|flags|comment|user|size'):
    page = site.pages[unquote(unquote(title))]
    revision = page.revisions(start=1192994591, end=1190146243, prop = prop)
    return revision

def revision_dataframe(title):
    revisions = get_revisions(title, prop='ids|timestamp|user|size')
    start = time.time()
    series = []
    for rev in revisions:
        rev['timestamp'] = calendar.timegm(rev['timestamp'])
        if 'user' not in rev:
            rev['user'] = ''
        if 'parentid' not in rev:
            rev['parentid'] = ''
        
                
        record = pd.Series(rev)
        series.append(record)
    if series:
        return pd.DataFrame(series, index = range(len(series)))
    else:
        return None
        
def collect_revisions(titles, nworkers = 4):
    results = []
    keys = []
    i = 0
    threads = [None] * nworkers
    flags = [True] * nworkers
    
    def target(index, flags, results, keys,title):
        try:
            df = revision_dataframe(title)
            f.write('word {} succeed\n'.format(title))
        except:
            f.write('word {} fail\n'.format(title))
            df = None
        if df is not None:
            results.append(df)
            keys.append(title)
            assert(len(results) == len(keys))
        flags[index] = True
    f = open('log_spider.txt','w')
    for title in titles:
        index = 0
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        t = threading.Thread(target = target, args = [index, flags, results, keys,title])
        threads[index] = t
        t.start()
        i += 1
        f.write('Processing word {}\n'.format(i))
        f.flush()
    for thread in threads:
        if thread:
            thread.join()
    return pd.concat(results, keys=keys)

In [14]:
#word_series = load_data("data/dump/hot_words_100000.pkl")
#words_list = word_series.index
#revision_df = collect_revisions(words_list)
#dump_data(revision_df,"data/dump/revision_records.pkl")

In [17]:
revision_df.to_csv("data/test.csv")

# Get metadata

language

media_num

categories

links

In [6]:
from ipywidgets import IntProgress

def get_metadata(title):
    meta = {}
    page = site.pages[unquote(unquote(title))]
    categories = list(page.categories(generator=False))
    langs = list(page.langlinks())
    links = list(page.links(generator=False))
    nimages = len(list(page.images(generator=False)))
    meta['languages'] = langs
    meta['links'] = links
    meta['categories'] = categories
    meta['image_num'] = nimages
    return meta

def collect_metas(titles, nworkers = 4):
    metas = {}
    i = 0
    threads = [None] * nworkers
    flags = [True] * nworkers
    
    def target(index, flags, title):
        try:
            meta = get_metadata(title)
        except:
            meta = None
        if meta is not None:
            metas[title] = meta
        flags[index] = True
    p =  IntProgress(max = 10000)  
    p.discription = "Running"
    display(p)
    for title in titles:
        index = 0
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        t = threading.Thread(target = target, args = [index, flags, title])
        threads[index] = t
        t.start()
        i += 1
        p.value += 1
        
    for thread in threads:
        if thread:
            thread.join()
    return metas

In [ ]:
word_series = load_data("data/dump/hot_words_100000.pkl")
words_list = word_series.index
delta = 10000
for j in range(1, 10):
    if os.path.isfile("data/dump/metadata_{}-{}.pkl".format(delta * j, delta * (j+1))):
        print("data/dump/metadata_{}-{}.pkl exists".format(delta * j, delta * (j+1)))
        continue
    print("data/dump/metadata_{}-{}.pkl".format(delta * j, delta * (j+1)))
    metas = collect_metas(words_list[delta * j: delta * (j+1)])
    dump_data(metas, "data/dump/metadata_{}-{}.pkl".format(delta * j, delta * (j+1)))

data/dump/metadata_10000-20000.pkl exists
data/dump/metadata_20000-30000.pkl exists
data/dump/metadata_30000-40000.pkl exists
data/dump/metadata_40000-50000.pkl exists
data/dump/metadata_50000-60000.pkl exists
data/dump/metadata_60000-70000.pkl exists
data/dump/metadata_70000-80000.pkl


IntProgress(value=0, max=10000)

data/dump/metadata_80000-90000.pkl


IntProgress(value=0, max=10000)

data/dump/metadata_90000-100000.pkl


IntProgress(value=0, max=10000)

99999

# Get page len

In [1]:
from pymongo import MongoClient
from mwclient import Site
import pandas
from tqdm import tqdm_notebook as tqdm
client = MongoClient()
db = client.get_database('wikipedia')

In [2]:
def get_text_of_hot_word(word):
    site = Site('en.wikipedia.org')
    try:
        text=site.pages[word].text()
        db.page_text.insert_one({'title':word,'text':text,'length':len(text)})
    except Exception as e:
        print(e)

In [3]:
def get_text_of_hot_words(hot_words):
    from mwclient import Site
    import multiprocessing.dummy
    p = multiprocessing.dummy.Pool(10)
    site = Site('en.wikipedia.org')
    exist_words=[page['title'] for page in db.page_text.find()]
    print('There are %d exist pages'%len(exist_words))
    hot_words=list(set(hot_words)-set(exist_words))
    for i in tqdm(range(0,len(hot_words),100)):
        try:
            p.map(get_text_of_hot_word, hot_words[i:i+100])
        except:
            continue

In [4]:
import pickle
import numpy

In [5]:
hot_words=pickle.load(open('data/dump/hot_words_100000.pkl','rb'))

In [6]:
hot_words=list(hot_words.index.values)

In [7]:
numpy.random.shuffle(hot_words)

In [8]:
get_text_of_hot_words(hot_words)

There are 65921 exist pages


The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%22".
The requested page title contains invalid characters: "%C5".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title

The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%27".
The requested page title contains invalid characters: "%27".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%C3".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%27".
The requested page title contains invalid characters: "%27".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title

The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%C3".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%26".
The requested page title contains invalid characters: "%28".
The requested page title

The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%2C".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%22".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%27".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%26".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title contains invalid characters: "%28".
The requested page title

...

The requested page title contains invalid characters: "%28".
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Colbert_report&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Economy_of_California&format=json&inprop=protection&continue=
The requested page title contains invalid characters: "%28".
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Plug-in_hybrid&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Field_Museum_of_Natural_History&format=json&inprop=protection&continue=
429 Client Error: Too Many Request

429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Armor&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Sting_%2528wrestler%2529&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=National_Honor_Society&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&rvprop=content%7Ctimestamp&prop=revisions&titles=Shopgirl&uiprop=blockinfo%7Chasmsg&meta=userinfo&rvlimit=1&format=json&rvdir=older&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasms

429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&rvprop=content%7Ctimestamp&prop=revisions&titles=Lana+Lang&uiprop=blockinfo%7Chasmsg&meta=userinfo&rvlimit=1&format=json&rvdir=older&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Ethnologue&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Elise_Neal&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=The_Silence_of_the_Lambs_%28film%29&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7C

429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Tommy_Steele&format=json&inprop=protection&continue=
The requested page title contains invalid characters: "%28".
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&rvprop=content%7Ctimestamp&prop=revisions&titles=Downblouse&uiprop=blockinfo%7Chasmsg&meta=userinfo&rvlimit=1&format=json&rvdir=older&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&rvprop=content%7Ctimestamp&prop=revisions&titles=Palace+of+Versailles&uiprop=blockinfo%7Chasmsg&meta=userinfo&rvlimit=1&format=json&rvdir=older&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&rvprop=content%7Ctimestamp&prop=revisions&titles=John+Amaechi&uiprop=blockinfo%7Chasmsg&meta=userinfo&rvlimit=1&format=json&rvdir=older&continue=
429 Client Er

The requested page title contains invalid characters: "%28".
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&rvprop=content%7Ctimestamp&prop=revisions&titles=Hamiltonian+path&uiprop=blockinfo%7Chasmsg&meta=userinfo&rvlimit=1&format=json&rvdir=older&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Mega_Man_%2528TV_series%2529&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Zlatan_Ibrahimovi%25C4%2587&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedia.org/w/api.php?action=query&prop=info&meta=userinfo&uiprop=blockinfo%7Chasmsg&titles=Eden&format=json&inprop=protection&continue=
429 Client Error: Too Many Requests for url: https://en.wikipedi